<a href="https://colab.research.google.com/github/SiheeNoh/DFC615/blob/master/DFC615_NLP_Task1_Movie_nsmc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 한국어 감정 분석 
  • 영화 리뷰를 바탕으로 긍정/부정을 예측 
  • NSMC 다운로드 :  https://github.com/e9t/nsmc


감성 분석
 나이브 베이즈 분류 모형을 이용하여 문서에 대한 감성 분석(sentiment analysis) 실행
  > 감성 분석이란 문서에 대해 좋다(positive) 혹은 나쁘다(negative)는 평가를 내리는 것
 샘플 데이터로는 github에 올려져 있는 네이버 영화 감상평에 대한 감성 분석 예제 이용
  > https://github.com/e9t/nsmc


Movie_nsmc

데이터 전처리 우선 데이터를 다운로드 받아서 읽기

In [1]:
%%time
!rm -f ko_data.txt ratings_train.txt ratings_test.txt

!wget -nc https://raw.githubusercontent.com/SiheeNoh/DFC615/master/ko_data.txt
!wget -nc https://raw.githubusercontent.com/SiheeNoh/DFC615/master/ratings_train.txt
!wget -nc https://raw.githubusercontent.com/SiheeNoh/DFC615/master/ratings_test.txt


--2020-06-25 07:32:11--  https://raw.githubusercontent.com/SiheeNoh/DFC615/master/ko_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622720 (608K) [text/plain]
Saving to: ‘ko_data.txt’

ko_data.txt         100%[===================>] 608.12K  --.-KB/s    in 0.06s   

2020-06-25 07:32:11 (10.6 MB/s) - ‘ko_data.txt’ saved [622720/622720]

--2020-06-25 07:32:13--  https://raw.githubusercontent.com/SiheeNoh/DFC615/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_tra

In [2]:
%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import plotly.express as px
import cufflinks as cf
cf.go_offline(connected=True)


유니코드로 인코딩하며 읽기 위해 codecs 패키지를 사용. 읽어들인 결과는 유니코드 문자열이 됨.

In [3]:
import codecs
with codecs.open('ratings_train.txt', 'r', encoding='utf-8') as f1:
    data_train = [line.split('\t') for line in f1.read().splitlines()]
    data_train = data_train[1:]   # header 제외


In [4]:
len(data_train)

150000

In [5]:
data_train[:1]

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']]

이 데이터는 번호, 내용, 평점으로 이루져 있으므로 내용을 X, 평점을 y로 저장.

In [6]:
from pprint import pprint
pprint(data_train[1])

['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1']


In [7]:
import numpy as np
X = list(zip(*data_train))[1]
y = np.array(list(zip(*data_train))[2], dtype=int)

다항 나이브 베이즈 모형으로 학습. 

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

model1 = Pipeline([
    ('vect', CountVectorizer()),
    ('mb', MultinomialNB()),
])

In [9]:
%%time
model1.fit(X, y)

CPU times: user 2.5 s, sys: 72.2 ms, total: 2.57 s
Wall time: 2.59 s


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [10]:
import codecs
with codecs.open("ratings_test.txt", encoding='utf-8') as f2:
    data_test = [line.split('\t') for line in f2.read().splitlines()]
    data_test = data_test[1:]   # header 제외

In [11]:
len(data_test)

50000

In [12]:
X_test = list(zip(*data_test))[1]
y_test = np.array(list(zip(*data_test))[2], dtype=int)

print(classification_report(y_test, model1.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83     24827
           1       0.84      0.81      0.82     25173

    accuracy                           0.83     50000
   macro avg       0.83      0.83      0.83     50000
weighted avg       0.83      0.83      0.83     50000



In [13]:
import codecs
with codecs.open("ko_data.txt", encoding='cp949') as f3:
    data_real = [line.split('\t') for line in f3.read().splitlines()]
    data_real = data_real[1:]   # header 제외

In [14]:
X_test_real = list(zip(*data_real))[1]
result = model1.predict(X_test_real)

In [15]:
idx = 0
for i in data_real :
  i.append(result[idx])
  idx+=1

In [16]:
str(data_real[0])

"['0', '정말 많이 울었던 영화입니다.', 1]"

결과값 저장(나이브 베이즈 방법)

In [17]:
ko_data_result = open("ko_data_result.txt", 'w')

idx = 0
for i in data_real :
  ko_data_result.write(str(data_real[idx][0])+'\t'+str(data_real[idx][2])+'\n')
  idx+=1
ko_data_result.close()


Tf-idf 방법

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

model2 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('mb', MultinomialNB()),
])

In [19]:
%%time
model2.fit(X, y)

CPU times: user 2.54 s, sys: 73.4 ms, total: 2.62 s
Wall time: 2.64 s


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [20]:
print(classification_report(y_test, model2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.84      0.83     24827
           1       0.84      0.81      0.83     25173

    accuracy                           0.83     50000
   macro avg       0.83      0.83      0.83     50000
weighted avg       0.83      0.83      0.83     50000



In [21]:
!pip install mecab-python3
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 17.1MB 243kB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.

형태소 분석기 사용

In [22]:
from konlpy.tag import Okt
pos_tagger = Okt()

def tokenize_pos(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc)]

In [23]:
model3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_pos)),
    ('mb', MultinomialNB()),
])

In [24]:
%%time
model3.fit(X, y)

CPU times: user 6min 12s, sys: 1.39 s, total: 6min 14s
Wall time: 6min


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize_pos at 0x7fae75de9378>,
                                 vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [25]:
print(classification_report(y_test, model3.predict(X_test)))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85     24827
           1       0.86      0.85      0.85     25173

    accuracy                           0.85     50000
   macro avg       0.85      0.85      0.85     50000
weighted avg       0.85      0.85      0.85     50000



(1,2)-gram 사용

In [26]:
model4 = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize_pos, ngram_range=(1, 2))),
    ('mb', MultinomialNB()),
])

In [27]:
%%time
model4.fit(X, y)

CPU times: user 9min 13s, sys: 1.54 s, total: 9min 14s
Wall time: 9min 6s


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize_pos at 0x7fae75de9378>,
                                 use_idf=True, vocabulary=None)),
                ('mb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [28]:
print(classification_report(y_test, model4.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87     24827
           1       0.87      0.86      0.87     25173

    accuracy                           0.87     50000
   macro avg       0.87      0.87      0.87     50000
weighted avg       0.87      0.87      0.87     50000



In [29]:
import codecs
with codecs.open("ko_data.txt", encoding='cp949') as f3:
    data_real_2 = [line.split('\t') for line in f3.read().splitlines()]
    data_real_2 = data_real_2[1:]   # header 제외

In [30]:
X_test_2 = list(zip(*data_real_2))[1]
result_ngram = model4.predict(X_test_2)

In [31]:
data_real_2[:3]

[['0', '정말 많이 울었던 영화입니다.'],
 ['1', '시간 낭비예요.'],
 ['2', '포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.']]

In [32]:
idx = 0
for i in data_real_2 :
  i.append(result_ngram[idx])
  idx+=1

결과 추출 : ngram (F1-score 값이 가장 좋음)

In [33]:
ko_data_result_ngram = open("ko_data_result_ngram.txt", 'w')

idx = 0
for i in data_real_2 :
  ko_data_result_ngram.write(str(data_real_2[idx][0])+'\t'+str(data_real_2[idx][2])+'\n')
  idx+=1
ko_data_result_ngram.close()